In [1]:
import numpy as np
import pandas as pd
from sys import executable, argv
from subprocess import check_output
import pctils as pc

import numpy as np
import pandas as pd
from sys import executable, argv
from subprocess import check_output
import pctils as pc
# SKLearn related imports
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier
import spacy
import hashlib

from tqdm import tqdm
from collections import Counter
from spacy.matcher import Matcher
from sklearn.metrics import accuracy_score
from nltk.tokenize import WordPunctTokenizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.preprocessing import OneHotEncoder


In [2]:
import nltk
nltk.download('stopwords')

import pandas as pd
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.base import TransformerMixin
import string
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pedrocondeco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:

# Transformers
def apply_tokenizer(data, tokenizer):
    lista_tok = [" ".join(tokenizer.tokenize(x)) for x in data]
    return lista_tok

def apply_lowercase(data):
    list_lower = [x.lower() for x in data]
    return list_lower

stopword_list = stopwords.words('english')
def apply_filter_stopwords(data, stopword_list):
    data_no_stopwords = []
    for x in data:
        aux = " ".join([w for w in x.split() if w not in stopword_list])
        data_no_stopwords.append(aux)
    return data_no_stopwords

def apply_filter_punct(data):
    data_no_punct = []
    for x in data:
        
        aux = "".join([w for w in x if w not in string.punctuation])
        data_no_punct.append(aux)

    return data_no_punct

def normalize_whitespace(data):
    return [re.sub(r"^\s+|\s+$|(?<=\s)\s*", "", text) for text in data]

def apply_stemmer(data, stemmer):
    list_tok = [WordPunctTokenizer().tokenize(x) for x in data]
    stems = [" ".join(list(map(stemmer.stem, y))) for y in list_tok]
    return stems

# creates a new column helpful_count/rates_count
class TransformLabels(TransformerMixin):
    def __init__(self, threshold = .5):
        self.helpful_count = 'helpful_count'
        self.rates_count = 'rates_count'
        self.threshold = threshold
        
    def isHelpful(self, helpful_count, rates_count):
        if rates_count == 0:
            return 0 # we don't know, not helpfull for now
        if helpful_count > rates_count:
            return 5 # five means bad, needs to be droped
        val = helpful_count/rates_count
        if val >= self.threshold:
            return 1
        return 0

    def transform(self, X, *_):
        _X = X.copy()

        label_array = []
        for h, r in zip(_X[self.helpful_count], _X[self.rates_count]):
            is_helpul = self.isHelpful(h,r)
            label_array += [is_helpul]

        out = pd.Series(label_array)
        return out
    
    def fit(self, *_):
        return self

class TextTransformer(TransformerMixin):
    def __init__(self, key):
        self.key = key

    def transform(self, X, *_):
        _X = X.copy()
        words = _X[self.key]
        features =  TfidfTransformer().fit_transform(words)
        return features
    
    def fit(self, *_):
        return self

class LazyTranformer(TransformerMixin):
    def __init__(self, key):
        self.key = key

    def transform(self, X, *_):
        _X = X.copy()
        return _X[self.key].to_frame()
    
    def fit(self, *_):
        return self

class Selector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        
    def fit(self, X, y=None):
        return self
    

class TextSelector(Selector):
    def transform(self, X):
        return X[self.key]

class NumberSelector(Selector):
    def transform(self, X):
        return X[[self.key]]

class TextCleanerTransformer(TransformerMixin):
    def __init__(self, column_key, tokenizer, lower=True, remove_punct=True, stopwords=[], stemmer=None):
        self.column_key = column_key
        self.tokenizer = tokenizer
        self.stemmer = stemmer
        self.lower = lower
        self.remove_punct = remove_punct
        self.stopwords = stopwords
    
    def clean_sentences(self, data):
                
        # Split sentence into list of words
        sentences_preprocessed = apply_tokenizer(data, self.tokenizer)
        # YOUR CODE HERE
        #raise NotImplementedError()
        
        # Lowercase
        if self.lower:
            sentences_preprocessed = apply_lowercase(sentences_preprocessed)
            # YOUR CODE HERE
            #raise NotImplementedError()

        if self.stopwords:
            sentences_preprocessed = apply_filter_stopwords(sentences_preprocessed,self.stopwords)
            # YOUR CODE HERE
            #raise NotImplementedError()
            
        # Remove punctuation
        if self.remove_punct:
            sentences_preprocessed = apply_filter_punct(sentences_preprocessed)
            # YOUR CODE HERE
            #raise NotImplementedError()
        
        # Normalize whitespace
        sentences_preprocessed = normalize_whitespace(sentences_preprocessed)
        # YOUR CODE HERE
        #raise NotImplementedError()
    
        # Stem words
        if self.stemmer:
            sentences_preprocessed = apply_stemmer(sentences_preprocessed,self.stemmer)
            # YOUR CODE HERE
            #raise NotImplementedError()

        return sentences_preprocessed

    def transform(self, X, *_):
        _X = X.copy()
        column_to_transform = _X[self.column_key]
        return self.clean_sentences(column_to_transform)


    def fit(self, X, y=None):
        return self

In [4]:
# configuration
#data_set = pc.getFile()  # gets path of tfile, you may replace with a string
data_set="/home/pedrocondeco/ldsa/group3/data/book_review_labelled_data.csv"


In [5]:
# configuration
#data_set_to_generalize = pc.getFile()  # gets path of tfile, you may replace with a string
data_set_to_generalize="/home/pedrocondeco/ldsa/group3/data/book_review_test_data_unlabelled.csv"

In [6]:
# configuration
label = "label" # TODO

In [7]:
# Load the dataset
df = pd.read_csv(data_set) 
df['reviewText'] = text_cleaner = TextCleanerTransformer(
    "reviewText",
    WordPunctTokenizer(),
    lower=True, 
    remove_punct=True, 
    stopwords=stopwords.words('english'),
    stemmer=SnowballStemmer("english"),
).transform(df)
df['summary'] = text_cleaner = TextCleanerTransformer(
    "summary",
    WordPunctTokenizer(),
    lower=True, 
    remove_punct=True, 
    stopwords=stopwords.words('english'),
    stemmer=SnowballStemmer("english"),
).transform(df)

# Load the stuff to generalize
df_to_generalize = pd.read_csv(data_set_to_generalize) 
df_to_generalize['reviewText'] = text_cleaner = TextCleanerTransformer(
    "reviewText",
    WordPunctTokenizer(),
    lower=True, 
    remove_punct=True, 
    stopwords=stopwords.words('english'),
    stemmer=SnowballStemmer("english"),
).transform(df_to_generalize)
df_to_generalize['summary'] = text_cleaner = TextCleanerTransformer(
    "summary",
    WordPunctTokenizer(),
    lower=True, 
    remove_punct=True, 
    stopwords=stopwords.words('english'),
    stemmer=SnowballStemmer("english"),
).transform(df_to_generalize)



# Show basic info
print(df.dtypes)            # all columns/types
print("Shape: ",df.shape)   # shape
df.head(10)

In [8]:
# Add label to df (without this it's not possible to train the model)
df[label] = TransformLabels(threshold=.5).fit_transform(df)  
# Show label distribution
pc.showColumnDistribution(df,label)

Number of entries:  49992

Distribution of the column:


0    27871
1    22121
Name: label, dtype: int64


Nan Values:  0


In [9]:
# Drop bad stuff?
# df = df[[df != 5]]

In [10]:
# Split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
train_label = df_train['label']
df_train.drop(label, inplace=True, axis=1)   # labels MUST NOT exist within train set

/home/pedrocondeco/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:

# Define features here, keep standard, concatenate _feature in the end
review_text_feature = Pipeline([
                                ('review_text_selector', TextSelector("reviewText")),
                                ('tfidf', TfidfVectorizer())
                            ])
summary_feature = Pipeline([
                                ('summary_selector', TextSelector("summary")),
                                ('tfidf', TfidfVectorizer())
                            ])
rating_feature = Pipeline([
                                ('rating_selector', NumberSelector("rating")),
                                ('onehot', OneHotEncoder())
                            ])
# Features
features = FeatureUnion([
                        ('summary_feature',summary_feature),
                        ('review_text_feature',review_text_feature)
                        ])

In [12]:
# Pipeline w/ model
model_pipeline = Pipeline([
                        ('features', features),
                        ('model',RandomForestClassifier(random_state = 42))
                    ])

In [13]:
# Baseline and print silly values
# returns model
def baseline(train_set, train_labels, test_set, model):
  # Train the classifier
  model.fit(train_set, train_labels)

  # Predict
  model_predictions = model.predict(test_set)

  # Print F1 stuff
  print(classification_report(test_set[label], model_predictions))

  _acc = np.mean(model_predictions == test_set[label])
  print('Accuracy: ' + str(_acc))

  return model

def stupidFunction(x):
    if x == 0:
        return "false"
    return "true"
def predictFromFileAndOutPutCSV(model, path):
    data_set_to_predict = pc.getFile()
    df = pd.read_csv(data_set_to_predict) # never modify
    predictions = model.predict(df)
    out = pd.DataFrame({'is_helpful': predictions})
    out['is_helpful'] =  out['is_helpful'].apply(lambda x: stupidFunction(x))
    out.to_csv(path, index=False)


In [14]:
model = baseline(df_train, train_label, df_test, model_pipeline)

              precision    recall  f1-score   support

           0       0.69      0.80      0.74      5590
           1       0.68      0.54      0.60      4409

    accuracy                           0.68      9999
   macro avg       0.68      0.67      0.67      9999
weighted avg       0.68      0.68      0.68      9999

Accuracy: 0.6845684568456846


In [15]:
predictFromFileAndOutPutCSV(model, 'done.csv')

KeyboardInterrupt: 